<div style="text-align: right; font-family: Arial;"><hr width="15%" align="right" style="margin-bottom: -0.8em;"><h4>Amazon Confidential</h4></div>

<p style="padding: 10px; border: 1px solid black;">
<img src="images/mlu-logo.png" alt="drawing" width="400"/> <br/>
<div style="background-image: linear-gradient(145deg, rgba(35, 47, 62, 1) 0%, rgba(51, 0, 102, 1) 40%, rgba(223, 42, 93, 1) 60%, rgba(124, 90, 237, 1) 85%, rgba(124, 232, 244, 1) 100%); padding: 20px; border-radius: 10px; text-align: center; margin-bottom: 30px;">
    <h1 style="color: white; margin: 0;">MLU: Agentic applications from concept to code</h1>
    <h2 style="color: white; margin-top: 15px;">Lab 1: Compare LLM with no tools vs tool-calling vs agents lab</h2>
</div>

<!-- Compact Lab Introduction with Activity/Challenge Explanation -->
<div style="background-color: #F8F9F9; padding: 15px; border-radius: 5px; margin: 20px 0;">
    <h4 style="color: #2E4053; margin-top: 0;">About This Lab</h4>
    <p>Throughout this lab, you will encounter two types of interactive elements:</p>
    <table style="width: 100%; border-collapse: collapse; margin: 15px 0;">
        <tr>
            <td style="text-align: center; padding: 10px; width: 50%;">
                <img src="images/mlu-activity.png" alt="Activity" width="125"/>
            </td>
            <td style="text-align: center; padding: 10px; width: 50%;">
                <img src="images/mlu-challenge.png" alt="Challenge" width="125"/>
            </td>
        </tr>
        <tr>
            <td style="text-align: center; padding: 10px; background-color: #EBF5FB;">
                <p>No coding is needed for an activity. You try to understand a concept, <br/>answer questions, or run a code cell.</p>
            </td>
            <td style="text-align: center; padding: 10px; background-color: #FEF9E7;">
                <p>Challenges are where you test your understanding by implementing something new or taking a short quiz.</p>
            </td>
        </tr>
    </table>
    <p>Please work through this notebook from top to bottom to avoid errors due to missing code or context.</p>
</div>

<!-- Table of Contents with All Section Levels -->
<div style="background-color: #f2f0fc; padding: 15px; border-radius: 5px; margin-bottom: 30px;">
    <h2 style="color: #2f1381; border-bottom: 1px solid #2f1381; padding-bottom: 5px;">Table of Contents</h2>
    <p><a href="#section1" style="color: #2f1381; font-weight: bold; text-decoration: none;">1. Setup and Configuration</a></p>
    <ul style="margin-top: 0; padding-left: 30px;">
        <li><a href="#section1-1" style="color: #2f1381; text-decoration: none;">1.1 Import dependencies</a></li>
        <li><a href="#section1-2" style="color: #2f1381; text-decoration: none;">1.2 Validate LLM model access</a></li>
    </ul>
    <p><a href="#section2" style="color: #2f1381; font-weight: bold; text-decoration: none;">2. Define the Amazon Bedrock Model for Inference</a></p>
    <p><a href="#section3" style="color: #2f1381; font-weight: bold; text-decoration: none;">3. Testing LLM Without Access to Any Tools</a></p>
    <p><a href="#section4" style="color: #2f1381; font-weight: bold; text-decoration: none;">4. Testing a Tool-Calling Workflow with Bedrock Converse API</a></p>
    <p><a href="#section5" style="color: #2f1381; font-weight: bold; text-decoration: none;">5. Building an Agentic Application Using Strands SDK</a></p>
    <ul style="margin-top: 0; padding-left: 30px;">
        <li><a href="#section5-1" style="color: #2f1381; text-decoration: none;">5.1 Setup custom tools</a></li>
    </ul>
    <p><a href="#section6" style="color: #2f1381; font-weight: bold; text-decoration: none;">6. Side-by-Side Testing</a></p>
    <p><a href="#section7" style="color: #2f1381; font-weight: bold; text-decoration: none;">7. Quizzes</a></p>
</div>

<!-- Section Header -->
<div id="section1" style="border-left: 5px solid #2f1381; padding-left: 15px; margin: 40px 0 20px 0;">
    <h2 style="color: #2f1381;">1. Setup and Configuration</h2>
</div>

Let's start by setting up our environment and creating the tools that both systems will use.

<!-- Subsection Header -->
<div id="section1-1" style="border-left: 3px solid #4C32E2; padding-left: 12px; margin: 25px 0 15px 15px;">
    <h3 style="color: #4C32E2;">1.1 Import dependencies</h3>
</div>

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore') 
!pip install -r requirements.txt --quiet

In [ ]:
# Core imports
from datetime import date
import requests
import json
import re
import boto3
import logging
from typing import Dict, Any
from requests.exceptions import RequestException

from strands import Agent, tool
from langchain_aws import ChatBedrockConverse

# Interactive widgets (we'll use these later)
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Helper utilities
from mlu_utils.helpers import validate_models_access

import warnings
warnings.filterwarnings('ignore')

<!-- Subsection Header -->
<div id="section1-2" style="border-left: 3px solid #4C32E2; padding-left: 12px; margin: 25px 0 15px 15px;">
    <h3 style="color: #4C32E2;">1.2 Validate LLM model access</h3>
</div>

As a first step we need to verify that the LLM models required in this lab are accessible. Let's do that now by using the helper function `validate_models_access` and provide the list of LLM models that we require for this lab. If the call to `validate_models_access` returns any model ids in the output list then you will need to go to the `Amazon Bedrock` console and enable access to the required models.

In [ ]:
validate_models_access(["amazon.nova-pro-v1:0"])

<!-- Section Header -->
<div id="section2" style="border-left: 5px solid #2f1381; padding-left: 15px; margin: 40px 0 20px 0;">
    <h2 style="color: #2f1381;">2. Define the Amazon Bedrock Model for Inference</h2>
</div>

Let's select the Amazon Bedrock model.

<!-- Tip Box -->
<div style="background-color: #E8F8F5; border-left: 5px solid #1ABC9C; padding: 15px; border-radius: 5px; margin: 20px 0;">
    <p style="color: #16A085; margin: 0;"><strong>Tip:</strong> Please opt for frugal practices when using Amazon Bedrock such as using smaller LLMs for simpler tasks and only reserving the use of the larger LLMs for more complex use cases.</p>
</div>

In [ ]:
# Constants for our models and configuration
MODEL_ID = "amazon.nova-pro-v1:0"
AWS_REGION = "us-east-1"  # Change if needed

# LangChain bedrock model for simple LLM calls
bedrock_llm = ChatBedrockConverse(
    model=MODEL_ID,
    temperature=0,
    max_tokens=None,
)

# Bedrock runtime client for tool calling
bedrock_runtime = boto3.client("bedrock-runtime", region_name=AWS_REGION)

print("✅ Bedrock models configured!")

<!-- Section Header -->
<div id="section3" style="border-left: 5px solid #2f1381; padding-left: 15px; margin: 40px 0 20px 0;">
    <h2 style="color: #2f1381;">3. Testing LLM Without Access to Any Tools</h2>
</div>

Let's first test out the LLM with simple prompting and see its limitations when asked about real-time information.

In [ ]:
from mlu_utils.lab1_utils.agents import NoToolsWithLLM

# Set it up
llm_no_tools = NoToolsWithLLM(model_id=MODEL_ID)

# Test queries that require real-time data
test_queries = [
    "What's the weather in New York right now?",
    "What's today's date?",
    "Should I bring an umbrella to Paris today?"
]

print("🔍 Testing Plain LLM (No Tools)")
print("=" * 50)

for i, query in enumerate(test_queries, 1):
    print(f"\n📝 Test {i}: {query}")
    print("-" * 40)
    
    response = llm_no_tools.process_query(query)
    print(f"Response: {response}")
    print()

<!-- Section Header -->
<div id="section4" style="border-left: 5px solid #2f1381; padding-left: 15px; margin: 40px 0 20px 0;">
    <h2 style="color: #2f1381;">4. Testing a Tool-Calling Workflow with Bedrock Converse API</h2>
</div>

Let's build a tool-calling system using Amazon Bedrock's Converse API that can intelligently choose and use tools, but follows a more structured approach than agents.

In [ ]:
import mlu_utils.lab1_utils.weather_tool as weather_tool
import mlu_utils.lab1_utils.date_tool as date_tool
from mlu_utils.lab1_utils.agents import ToolCallingLLM

# System prompt for tool-calling
system_prompt = """
You are a helpful assistant that can provide weather information and current date.

You have access to these tools:
1. Weather_Tool: Get current weather for any location (requires latitude,longitude coordinates)
2. Date_Tool: Get today's date

When users ask about weather, you should:
- Use the Weather_Tool with appropriate coordinates
- Common city coordinates you can use:
  - New York: 40.7128,-74.0060
  - London: 51.5074,-0.1278
  - Paris: 48.8566,2.3522
  - Tokyo: 35.6762,139.6503
  - Sydney: -33.8688,151.2093
  - Berlin: 52.5200,13.4050
  - Mumbai: 19.0760,72.8777
  - Seattle: 47.6062,-122.3321

When users ask about the current date, use the Date_Tool.

Always provide helpful and accurate responses based on the tool results.
"""

# Create the tool-calling system
tool_calling_system = ToolCallingLLM(
    model_id=MODEL_ID,
    system_prompt = system_prompt, 
    tools=[weather_tool, date_tool]
)
print("✅ Tool-calling system created!")

In [ ]:
# Test 1: Simple weather query
query1 = "What's the weather in Paris?"

print("🧪 Test 1: Simple weather query")
print(f"Query: {query1}")
print("=" * 50)
response = tool_calling_system.process_query(query1)
print(f"\n📋 Final Response: {response}")
print("\n" + "=" * 80)

In [ ]:
# Test 2: Date query
query2 = "What's today's date?"

print("🧪 Test 2: Date query")
print(f"Query: {query2}")
print("=" * 50)
response = tool_calling_system.process_query(query2)
print(f"\n📋 Final Response: {response}")
print("\n" + "=" * 80)

In [ ]:
# Test 3: Natural language query
query3 = 'Should I bring an umbrella when I go out in New York today?'

print("🧪 Test 3: Natural language query")
print(f"Query: {query3}")
print("=" * 50)
response = tool_calling_system.process_query(query3)
print(f"\n📋 Final Response: {response}")
print("\n" + "=" * 80)

<!-- Section Header -->
<div id="section5" style="border-left: 5px solid #2f1381; padding-left: 15px; margin: 40px 0 20px 0;">
    <h2 style="color: #2f1381;">5. Building an Agentic Application Using Strands SDK</h2>
</div>

Now let's build an intelligent agent using Strands SDK that provides more flexible reasoning and autonomous decision-making capabilities.

<!-- Subsection Header -->
<div id="section5-1" style="border-left: 3px solid #4C32E2; padding-left: 12px; margin: 25px 0 15px 15px;">
    <h3 style="color: #4C32E2;">5.1 Setup custom tools</h3>
</div>

Both the tool-calling system and the agent will have access to the same tools. The difference is in **how they decide which tool to use.**

In [ ]:
# Tool 1: Get current date - Strands version
@tool
def get_curr_date(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date math should occur \
    outside this function."""
    return str(date.today())

# Tool 2: Get weather information - Strands version
@tool 
def get_weather(location_coords: str) -> str:
    """Get current weather for a location using latitude,longitude coordinates.
    Input should be in format: 'latitude,longitude' (e.g., '40.7128,-74.0060' for New York)"""
    try:
        # https://open-meteo.com/en/docs
        endpoint = "https://api.open-meteo.com/v1/forecast"
        latitude, longitude = location_coords.split(",")
        latitude = float(latitude.strip())
        longitude = float(longitude.strip())
        params = {"latitude": latitude, "longitude": longitude, "current_weather": True}

        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        weather_data = response.json()
        return json.dumps({"weather_data": weather_data})
    except Exception as e:
        return json.dumps({"error": str(type(e).__name__), "message": str(e)})

print("✅ Tools created successfully!")
print("📅 Date tool: get_curr_date() / Date_Tool")
print("🌤️ Weather tool: get_weather() / Weather_Tool")

<!-- Important Note -->
<div style="background-color: #FDEDEC; border-left: 5px solid #E74C3C; padding: 15px; border-radius: 5px; margin: 20px 0;">
    <p style="color: #C0392B; margin: 0;"><strong>Important:</strong> If you find that it is taking too long for any kind of response, it might be due to the throttling limits in Amazon Bedrock. Please stop the execution and re-run the cell.</p>
</div>

In [ ]:
from mlu_utils.lab1_utils.agents import StrandsAgent

# Create the intelligent agent
tools = [get_curr_date, get_weather]
strands_agent = StrandsAgent(model_id=MODEL_ID, tools=tools)
print("✅ Strands Agent created!")

<!-- Activity Box -->
<div style="background-color: #EBF5FB; border-left: 5px solid #3498DB; padding: 15px; border-radius: 5px; margin: 20px 0; display: flex; align-items: flex-start;">
    <div style="flex: 0 0 60px; margin-right: 15px;">
        <img src="images/mlu-activity.png" alt="Activity" width="200" style="max-width: 100%; height: auto;">
    </div>
    <div style="flex: 1;">
        <h4 style="color: #2874A6; margin-top: 0;">Activity: Test the Strands Agent</h4>
        <p>Run the same queries and observe how the agent <strong>reasons autonomously</strong> about what you want. Notice the planning and execution steps!</p>
    </div>
</div>

In [ ]:
# Test 1: Complex contextual query
query2 = "I'm planning a picnic in Central Park tomorrow. What should I prepare for?"

print("🧪 Test 2: Complex contextual query")
print(f"Query: {query2}")
print("=" * 50)
response = strands_agent.process_query(query2)
print(f"\n📋 Final Answer: {response}")
print("\n" + "=" * 80)

In [ ]:
# Test 2: Multi-step reasoning query
query3 = "Is it a good day to visit the Eiffel Tower right now?"

print("🧪 Test 3: Multi-step reasoning query")
print(f"Query: {query3}")
print("=" * 50)
response = strands_agent.process_query(query3)
print(f"\n📋 Final Answer: {response}")
print("\n" + "=" * 80)

<!-- Section Header -->
<div id="section6" style="border-left: 5px solid #2f1381; padding-left: 15px; margin: 40px 0 20px 0;">
    <h2 style="color: #2f1381;">6. Side-by-Side Testing</h2>
</div>

Let's compare all three systems directly with challenging queries to see the differences in capability using a visual comparison table.

<!-- Important Note -->
<div style="background-color: #FDEDEC; border-left: 5px solid #E74C3C; padding: 15px; border-radius: 5px; margin: 20px 0;">
    <p style="color: #C0392B; margin: 0;"><strong>Important:</strong> If you find that it is taking too long for any kind of response, it might be due to the throttling limits in Amazon Bedrock. Please stop the execution and re-run the cell.</p>
</div>

<!-- Activity Box -->
<div style="background-color: #EBF5FB; border-left: 5px solid #3498DB; padding: 15px; border-radius: 5px; margin: 20px 0; display: flex; align-items: flex-start;">
    <div style="flex: 0 0 60px; margin-right: 15px;">
        <img src="images/mlu-activity.png" alt="Activity" width="200" style="max-width: 100%; height: auto;">
    </div>
    <div style="flex: 1;">
        <h4 style="color: #2874A6; margin-top: 0;">Activity: Side-by-Side Visual Comparison</h4>
        <p>Run the cells below to see visual comparisons between all three approaches. Notice how each system handles the same queries differently!</p>
    </div>
</div>

In [ ]:
from mlu_utils.lab1_utils.comparison_helper import compare_all_systems
# Test: Contextual weather query
compare_all_systems(
    query = "Should I pack warm clothes for my Berlin trip?",
    test_plain_llm=llm_no_tools, 
    tool_calling_system=tool_calling_system, 
    strands_agent=strands_agent)

<!-- Activity Box -->
<div style="background-color: #EBF5FB; border-left: 5px solid #3498DB; padding: 15px; border-radius: 5px; margin: 20px 0; display: flex; align-items: flex-start;">
    <div style="flex: 0 0 60px; margin-right: 15px;">
        <img src="images/mlu-activity.png" alt="Activity" width="200" style="max-width: 100%; height: auto;">
    </div>
    <div style="flex: 1;">
        <h4 style="color: #2874A6; margin-top: 0;">Activity: Compare all three techniques</h4>
        <p>Try other prompts to see if and how agentic applications perform better due to their ability to plan and execute tools </p>
    </div>
</div>

In [ ]:
# Try your own queries here!

query = "<Add your query here>"

# compare_all_systems(
#     query = query,
#     test_plain_llm=llm_no_tools, 
#     tool_calling_system=tool_calling_system, 
#     strands_agent=strands_agent)

<!-- Section Header -->
<div id="section7" style="border-left: 5px solid #2f1381; padding-left: 15px; margin: 40px 0 20px 0;">
    <h2 style="color: #2f1381;">7. Quizzes</h2>
</div>

Well done on completing the lab! Now, it's time for a brief knowledge assessment.

<!-- Challenge Box -->
<div style="background-color: #FEF9E7; border-left: 5px solid #F1C40F; padding: 15px; border-radius: 5px; margin: 20px 0; display: flex; align-items: flex-start;">
    <div style="flex: 0 0 60px; margin-right: 15px;">
        <img src="images/mlu-challenge.png" alt="Challenge" width="200" style="max-width: 100%; height: auto;">
    </div>
    <div style="flex: 1;">
        <h4 style="color: #B7950B; margin-top: 0;">Challenge: Knowledge assessment</h4>
        <p>Answer the following questions to test your understanding of agents.</p>
    </div>
</div>

In [ ]:
from mlu_utils.quiz import Quiz
from mlu_utils.quiz_questions import lab1a_question1, lab1a_question2, lab1b_question1, lab1b_question2

Quiz(lab1a_question1).display()
Quiz(lab1a_question2).display()
Quiz(lab1b_question1).display()
Quiz(lab1b_question2).display()

<div style="background-color: #f2f0fc; padding: 15px; border-radius: 5px; margin: 30px 0;">
    <h3 style="color: #2f1381; border-bottom: 1px solid #2f1381; padding-bottom: 5px;">Conclusion</h3>
    <p style="color: #2f1381;">In this lab, you have:</p>
    <ul>
        <li style="color: #2f1381;">Explored how LLMs without tools can hallucinate depending on the type of user query</li>
        <li style="color: #2f1381;">Tools are almost mandatory for real-life use-cases</li>
        <li style="color: #2f1381;">Explored how to integrate custom tools with strands agents</li>
        <li style="color: #2f1381;">Created tools like date tool and weather tool invoking API</li>
        <li style="color: #2f1381;">Explored how to integrate custom tools with LLMs like Nova Pro</li>
    </ul>
    <h4 style="color: #2f1381; margin-top: 15px;">Additional Resources</h4>
    <ul>
        <li><a href="https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/tools_overview/">Strands Tools Documentation</a></li>
        <li><a href="https://strandsagents.com/latest/documentation/docs/user-guide/quickstart/">Strands Agents Documentation</a></li>
        <li><a href="https://docs.aws.amazon.com/nova/latest/userguide/tool-use.html">Nova Pro tool use Documentation</a></li>
    </ul>
</div>

<p style="padding: 10px; border: 1px solid black;">
<img src="images/mlu-logo.png" alt="drawing" width="400"/> <br/>

# Thank you!